# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-11-09 17:07:55] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-11-09 17:07:55] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-11-09 17:07:55] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-11-09 17:07:57] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-11-09 17:08:05] WARNING server_args.py:1186: Attention backend not explicitly specified. Use fa3 backend by default.


...

Capturing batches (bs=112 avail_mem=68.26 GB):   5%|▌         | 1/20 [00:00<00:03,  5.43it/s]

Capturing batches (bs=72 avail_mem=68.23 GB):  30%|███       | 6/20 [00:00<00:01, 12.12it/s]

Capturing batches (bs=24 avail_mem=68.20 GB):  60%|██████    | 12/20 [00:00<00:00, 19.05it/s]

Capturing batches (bs=8 avail_mem=68.19 GB):  75%|███████▌  | 15/20 [00:00<00:00, 18.89it/s] 

Capturing batches (bs=1 avail_mem=76.21 GB): 100%|██████████| 20/20 [00:01<00:00, 15.89it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Vitor. I am from Brazil and I am 22 years old. I have a very strong desire to go to the United States for a 3 month summer vacation. The reason for this is that I have heard that the US is the most beautiful place to live in the world. The first question I need to ask myself is "If I go to the US, what would my first impression of the US be? "
Would you be able to tell me what the first impression of the US would be?

Certainly! If you were to visit the United States for a 3-month summer vacation, your first impressions of the country
Prompt: The president of the United States is
Generated text:  from which country? France

The president of the United States is a member of the executive branch of the government of the United States, and France is the country where the president of the United States is from. The president is elected by the people of the United States and serves a term of four years. France is the sixth most populous country in 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [job title] at [company name], and I'm excited to meet you and learn more about you. What

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as the City of Light, and is the largest city in the country and the seat of the French government. It is located on the Seine River and is home to many of France's most famous landmarks, including the Eiffel Tower, the Louvre Museum, and the Notre-Dame Cathedral. Paris is also known for its rich cultural heritage, including the Opéra Garnier, the Musée d'Orsay, and the Champs-Élysées. The city is also home to many international institutions and organizations, including the French Academy of Sciences and the European Parliament. Paris is a vibrant and dynamic

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and decision-making processes.

2. Greater emphasis on ethical considerations: As AI becomes more integrated with human intelligence, there will be a greater emphasis on ethical considerations, such as privacy, fairness, and accountability.

3. Development of more advanced AI: AI will continue to evolve and improve, with new algorithms and techniques being developed to make AI more efficient and effective.

4. Increased use of AI in healthcare: AI is likely to be used more extensively in



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I am a [occupation]. I live in [city]. I enjoy [something that interests me]. I am [age]. I am a [occupation]. I enjoy [something that interests me]. I am [age]. I am a [occupation]. I enjoy [something that interests me]. I am [age]. I am a [occupation]. I enjoy [something that interests me]. I am [age]. I am a [occupation]. I enjoy [something that interests me]. I am [age]. I am a [occupation]. I enjoy [something that interests me]. I am [age]. I am a [

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, which is known for its iconic architecture, vibrant culture, and rich history. It has been the seat of the French government and the country's political capital since 1871, and has a population of over 2.7 million people. Paris is a major center for the arts, fashion, and food, and is home to m

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

]

 and

...

 the

 effectiveness

 of

 treatments

,

In [6]:
llm.shutdown()